# Regression

In [33]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from sklearn.preprocessing import StandardScaler

from torchcp.regression.predictors import SplitPredictor,CQR,ACI
from torchcp.regression.loss import QuantileLoss
from torchcp.utils import fix_randomness
import pandas as pd

In [34]:
def train(model, device, epoch, train_data_loader, criterion, optimizer):
    for index, (tmp_x, tmp_y) in enumerate(train_data_loader):
        outputs = model(tmp_x.to(device))
        loss = criterion(outputs, tmp_y.unsqueeze(dim=1).to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [35]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
fix_randomness(seed=1)

## Data

In [36]:
attrib = pd.read_csv("/data/lab/STA303-Exercise02/attributes.csv", delim_whitespace=True)
data = pd.read_csv('/data/lab/STA303-Exercise02/communities.data', names=attrib['attributes'])
data = data.drop(columns=['state', 'county',
                          'community', 'communityname',
                          'fold'], axis=1)
data = data.replace('?', np.nan)

# Impute mean values for samples with missing values
data['OtherPerCap'] = data['OtherPerCap'].astype("float")
mean_value = data['OtherPerCap'].mean()
data['OtherPerCap'].fillna(value=mean_value, inplace=True)
data = data.dropna(axis=1)
X = data.iloc[:, 0:100].values
y = data.iloc[:, 100].values

X = X.astype(np.float32)
y = y.astype(np.float32)

In [38]:
X.shape

(1994, 100)

In [39]:
y.shape

(1994,)

In [29]:
indices = np.arange(X.shape[0])
np.random.shuffle(indices)
split_index1 = int(len(indices) * 0.4)
split_index2 = int(len(indices) * 0.6)
part1, part2, part3 = np.split(indices, [split_index1, split_index2])
scalerX = StandardScaler()
scalerX = scalerX.fit(X[part1, :])
train_dataset = TensorDataset(torch.from_numpy(scalerX.transform(X[part1, :])), torch.from_numpy(y[part1]))
cal_dataset = TensorDataset(torch.from_numpy(scalerX.transform(X[part2, :])), torch.from_numpy(y[part2]))
test_dataset = TensorDataset(torch.from_numpy(scalerX.transform(X[part3, :])), torch.from_numpy(y[part3]))

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True, pin_memory=True)
cal_data_loader = torch.utils.data.DataLoader(cal_dataset, batch_size=100, shuffle=False, pin_memory=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, pin_memory=True)

epochs = 100
alpha = 0.1

## NonLinNet

In [30]:
class NonLinearNet(nn.Module):
    def __init__(self, in_shape, out_shape, hidden_size, dropout):
        super(NonLinearNet, self).__init__()
        self.hidden_size = hidden_size
        self.in_shape = in_shape
        self.out_shape = out_shape
        self.dropout = dropout
        self.base_model = nn.Sequential(
            nn.Linear(self.in_shape, self.hidden_size),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_size, self.hidden_size),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_size, self.out_shape),
        )

    def forward(self, x):
        return self.base_model(x)

## MSELoss

In [31]:
model = NonLinearNet(X.shape[1], 1, 64, 0.5).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(epochs):
    train(model, device, epoch, train_data_loader, criterion, optimizer)
    
model.eval()
predictor = SplitPredictor(model)
predictor.calibrate(cal_data_loader, alpha)
print(predictor.evaluate(test_data_loader))

{'Coverage_rate': 0.9110275506973267, 'Average_size': 0.5448101758956909}


## QuantileLoss

In [32]:
quantiles = [alpha / 2, 1 - alpha / 2]
model = NonLinearNet(X.shape[1], 2, 64, 0.5).to(device)
criterion = QuantileLoss(quantiles)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(epochs):
    train(model, device, epoch, train_data_loader, criterion, optimizer)

model.eval()
predictor = CQR(model)
predictor.calibrate(cal_data_loader, alpha)
print(predictor.evaluate(test_data_loader))

{'Coverage_rate': 0.9360902309417725, 'Average_size': 0.48810264468193054}
